Entropy is the qualtification of level of disorder. 

a High Entropy means high disorder.

In information theory, the less entropy you have the more information you have.

At initialization, a ANN is at high state of entropy. When training, the ANN weights are at low entropy whichmeans they managed to learn information from data.

the formula and link between entropy and information is :

Information = - log2(P) // with P the probability of an event occuring
// if P=1 the evnent is certain and we will  no gain any Information from it so the Information = 0
// P = 0 is impossible and the Information resulting is infinite, which mean it will requestion our understanding and world modelm

We use the log2 to compute the information in bit. if p=0.5 ==> -log2(0.5)=1 , we need one bit to encode this message.


Also information is a measure of difference between 2 entropies.
Entropy can also tell us what is the maximum information we can learn about a system.
The second law of thermodynamic says that entropy is increasing.

Cross entropy = -SUM(Pk.log2(Pk))

In [1]:
import numpy as np

def entropy(probabilities, base=2):
    """
    Calculate the entropy of a distribution given its probabilities.

    Parameters:
    - probabilities: array-like, list of probabilities of outcomes.
    - base: logarithm base, default is 2 for entropy in bits.

    Returns:
    - Entropy of the distribution.
    """
    probabilities = np.array(probabilities)  # Ensure input is numpy array
    # Filter out zero probabilities to avoid log(0)
    probabilities = probabilities[probabilities > 0]
    return -np.sum(probabilities * np.log(probabilities) / np.log(base))

# Examples

# Fair coin
fair_coin = [0.5, 0.5]
entropy_fair_coin = entropy(fair_coin)

# Biased coin
biased_coin = [0.75, 0.25]
entropy_biased_coin = entropy(biased_coin)

entropy_fair_coin, entropy_biased_coin


(1.0, 0.8112781244591328)

In [2]:
import numpy as np

def joint_entropy(distribution, base=2):
    """
    Calculate the joint entropy of a 2-dimensional distribution.

    Parameters:
    - distribution: 2D array-like, joint distribution probabilities of outcomes.
    - base: logarithm base, default is 2 for entropy in bits.

    Returns:
    - Joint entropy of the distribution.
    """
    distribution = np.array(distribution)
    # Filter out zero probabilities to avoid log(0)
    non_zero_probs = distribution[distribution > 0]
    return -np.sum(non_zero_probs * np.log(non_zero_probs) / np.log(base))

def conditional_entropy(joint_distribution, marginal_y, base=2):
    """
    Calculate the conditional entropy H(X|Y) given a joint distribution of X and Y and the marginal distribution of Y.

    Parameters:
    - joint_distribution: 2D array-like, joint distribution probabilities of X and Y.
    - marginal_y: array-like, marginal probabilities of Y.
    - base: logarithm base, default is 2 for entropy in bits.

    Returns:
    - Conditional entropy of X given Y.
    """
    joint_distribution = np.array(joint_distribution)
    marginal_y = np.array(marginal_y)
    conditional_probs = joint_distribution / marginal_y
    # Filter out zero probabilities to avoid division by zero and log(0)
    valid_probs = conditional_probs[marginal_y > 0]
    return -np.sum(joint_distribution[marginal_y > 0] * np.log(valid_probs) / np.log(base))

# Example: Joint and Conditional Entropy
# Let's consider a simple example with a joint probability distribution
joint_distribution = np.array([[0.1, 0.2], [0.3, 0.4]])  # P(X,Y)
marginal_y = np.sum(joint_distribution, axis=0)  # Sum over X to get P(Y)

# Calculate Joint Entropy H(X,Y)
joint_entropy_value = joint_entropy(joint_distribution)

# Calculate Conditional Entropy H(X|Y)
conditional_entropy_value = conditional_entropy(joint_distribution, marginal_y)

joint_entropy_value, conditional_entropy_value


(1.8464393446710154, 0.8754887502163471)

In [3]:
import torch

def mutual_information(joint_distribution, marginal_x, marginal_y, base=2):
    """
    Calculate the mutual information I(X;Y) given a joint distribution of X and Y and their marginal distributions.

    Parameters:
    - joint_distribution: 2D Tensor, joint distribution probabilities of X and Y.
    - marginal_x: 1D Tensor, marginal probabilities of X.
    - marginal_y: 1D Tensor, marginal probabilities of Y.
    - base: logarithm base, default is 2 for entropy in bits.

    Returns:
    - Mutual information of X and Y.
    """
    # Ensure inputs are tensors
    joint_distribution = torch.tensor(joint_distribution, dtype=torch.float)
    marginal_x = torch.tensor(marginal_x, dtype=torch.float).unsqueeze(1)  # Make marginal_x column vector
    marginal_y = torch.tensor(marginal_y, dtype=torch.float).unsqueeze(0)  # Make marginal_y row vector

    # Calculate the product of marginals for each pair (x, y)
    product_of_marginals = marginal_x * marginal_y

    # Calculate mutual information
    # Use where to avoid log(0) and division by 0
    valid_joint_probs = joint_distribution > 0
    mutual_info = torch.where(valid_joint_probs,
                              joint_distribution * torch.log2(joint_distribution / product_of_marginals),
                              torch.zeros_like(joint_distribution))
    
    return mutual_info.sum()

# Example: Mutual Information
joint_distribution = torch.tensor([[0.1, 0.2], [0.3, 0.4]])
marginal_x = torch.sum(joint_distribution, dim=1)  # Sum over Y to get P(X)
marginal_y = torch.sum(joint_distribution, dim=0)  # Sum over X to get P(Y)

# Calculate Mutual Information I(X;Y)
mutual_information_value = mutual_information(joint_distribution, marginal_x, marginal_y)

mutual_information_value


/var/folders/l4/bv3z08h91lggkmhcp2n7yzh80000gn/T/ipykernel_24158/159531598.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  joint_distribution = torch.tensor(joint_distribution, dtype=torch.float)
/var/folders/l4/bv3z08h91lggkmhcp2n7yzh80000gn/T/ipykernel_24158/159531598.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  marginal_x = torch.tensor(marginal_x, dtype=torch.float).unsqueeze(1)  # Make marginal_x column vector
/var/folders/l4/bv3z08h91lggkmhcp2n7yzh80000gn/T/ipykernel_24158/159531598.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(s

tensor(0.0058)